In [17]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv("D:/machine learning/datasets/agri.csv")
data.head(10)

,state,district,market,commodity,variety,arrival_date,min_price,max_price,modal_price
0,Andaman and Nicobar,South Andaman,Port Blair,Amaranthus,Amaranthus,04/03/2019,6000,8000,7000
1,Andaman and Nicobar,South Andaman,Port Blair,Banana - Green,Banana - Green,04/03/2019,4500,5500,5000
2,Andaman and Nicobar,South Andaman,Port Blair,Bhindi(Ladies Finger),Bhindi,04/03/2019,6000,8000,7000
3,Andaman and Nicobar,South Andaman,Port Blair,Bitter gourd,Other,04/03/2019,6000,8000,7000
4,Andaman and Nicobar,South Andaman,Port Blair,Black pepper,Other,04/03/2019,110000,130000,120000
5,Andaman and Nicobar,South Andaman,Port Blair,Bottle gourd,Other,04/03/2019,3000,4000,3500
6,Andaman and Nicobar,South Andaman,Port Blair,Brinjal,Other,04/03/2019,7000,8000,7500
7,Andaman and Nicobar,South Andaman,Port Blair,Cabbage,Other,04/03/2019,6000,7000,6500
8,Andaman and Nicobar,South Andaman,Port Blair,Carrot,Other,04/03/2019,9000,11000,10000
9,Andaman and Nicobar,South Andaman,Port Blair,Cauliflower,Other,04/03/2019,10000,12000,11000


In [3]:
data = data.dropna()

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2238 entries, 0 to 2237
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state         2238 non-null   object 
 1   district      2238 non-null   object 
 2   market        2238 non-null   object 
 3   commodity     2238 non-null   object 
 4   variety       2238 non-null   object 
 5   arrival_date  2238 non-null   object 
 6   min_price     2238 non-null   float64
 7   max_price     2238 non-null   float64
 8   modal_price   2238 non-null   float64
dtypes: float64(3), object(6)
memory usage: 174.8+ KB


In [4]:
data['min_price'] = data['min_price']/100
data['max_price'] = data['max_price']/100
data['modal_price'] = data['modal_price'] / 100

In [5]:
data.head()

,state,district,market,commodity,variety,arrival_date,min_price,max_price,modal_price
0,Andaman and Nicobar,South Andaman,Port Blair,Amaranthus,Amaranthus,04/03/2019,60.0,80.0,70.0
1,Andaman and Nicobar,South Andaman,Port Blair,Banana - Green,Banana - Green,04/03/2019,45.0,55.0,50.0
2,Andaman and Nicobar,South Andaman,Port Blair,Bhindi(Ladies Finger),Bhindi,04/03/2019,60.0,80.0,70.0
3,Andaman and Nicobar,South Andaman,Port Blair,Bitter gourd,Other,04/03/2019,60.0,80.0,70.0
4,Andaman and Nicobar,South Andaman,Port Blair,Black pepper,Other,04/03/2019,1100.0,1300.0,1200.0


In [13]:
X1 = data.iloc[:,:8]

In [16]:
Y1 = data['modal_price']

In [49]:
X = data[['commodity','min_price','max_price']]
Y = data['modal_price']
cat_mask = (X.dtypes==object)
cat_cols = X.columns[cat_mask].tolist()
le = LabelEncoder()
X[cat_cols] = X[cat_cols].apply(lambda x:le.fit_transform(x))

C:\Users\Admin\AppData\Local\Temp\ipykernel_2436\1597142809.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_cols] = X[cat_cols].apply(lambda x:le.fit_transform(x))


In [50]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [53]:
def xgb_model():
    st = StandardScaler()
    xgb_reg = xgb.XGBRegressor()
    steps = [('scaler',st),('model',xgb_reg)]
    xgb_pipeline = Pipeline(steps)
    
    param = {
        'model__subsample' : np.arange(0.05,1.05),
        'model__max_depth': np.arange(3,20,1),
        'model__colsample_bytree':np.arange(.1,1.05,.05),
        'model__learning_rate': np.arange(0,1,.1)
    }
    rand = RandomizedSearchCV(estimator=xgb_pipeline,param_distributions = param,n_iter=3,scoring='neg_mean_squared_error',cv=4)
    rand.fit(X_train,Y_train)
    model = rand.best_estimator_
    return model

In [9]:
model = xgb_model()

In [24]:
model.fit(X_train,Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.7500000000000002,
                              enable_categorical=False, gamma=0, gpu_id=-1,
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.1, max_delta_step=0, max_depth=4,
                              min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=8, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=0.05,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [25]:
model.score(X_test,Y_test)

0.9765838343922122

In [52]:
model1 = xgb_model()

In [53]:
model1.fit(X_train,Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.3500000000000001,
                              enable_categorical=False, gamma=0, gpu_id=-1,
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.30000000000000004,
                              max_delta_step=0, max_depth=17,
                              min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=8, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=0.05,
                              tree_method='exact', validate_parameters=1,
             

In [54]:
model1.score(X_test,Y_test)

0.42613875719748584

In [57]:
import pickle

In [28]:
regressor = deploy['model']
le_com = deploy['le_c']

In [30]:
y_pred = regressor.predict(X_test)

In [32]:
Y_test

1753    5500
367     2550
1247    3525
849     3700
1748     700
        ... 
1801    1400
178     6300
471     2500
1456    1770
2162     750
Name: modal_price, Length: 448, dtype: int64

In [18]:
rf = RandomForestRegressor()

In [20]:
cat_mask = (X1.dtypes==object)
cat_cols = X1.columns[cat_mask].tolist()
le = LabelEncoder()
X1[cat_cols] = X1[cat_cols].apply(lambda x:le.fit_transform(x))

In [21]:
rf.fit(X1,Y1)

RandomForestRegressor()

In [22]:
rf.feature_importances_

array([3.04186605e-03, 4.31267865e-04, 4.54409101e-04, 1.34281791e-03,
       3.40378241e-04, 0.00000000e+00, 2.63784109e-01, 7.30605152e-01])

In [23]:
from sklearn.feature_selection import RFE

In [24]:
rfe = RFE(estimator=rf,n_features_to_select = 4, step = 5, verbose=1 )

In [25]:
rfe.fit(X1,Y1)

Fitting estimator with 8 features.


RFE(estimator=RandomForestRegressor(), n_features_to_select=4, step=5,
    verbose=1)

In [26]:
rfe.support_

array([ True, False, False,  True, False, False,  True,  True])

In [27]:
Xt = data[['state','commodity']]
cat_mask = (Xt.dtypes==object)
cat_cols = Xt.columns[cat_mask].tolist()
le = LabelEncoder()
Xt[cat_cols] = Xt[cat_cols].apply(lambda x:le.fit_transform(x))

c:\users\admin\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [29]:
X_t,X_te,Y_t,Y_te = train_test_split(Xt,Y1,test_size=0.2)

In [30]:
rf.fit(X_t,Y_t)

RandomForestRegressor()

In [31]:
rf.score(X_te,Y_te)

0.6158018774112052

In [32]:
rf.predict(X_te)

array([4.37245893e+01, 5.09397978e+00, 2.69765534e+01, 2.31323056e+01,
       1.20584008e+01, 5.87317358e+00, 4.20586295e+00, 5.09397978e+00,
       1.16464213e+01, 5.49301689e+01, 2.29483333e+01, 5.14487316e+01,
       1.52815952e+01, 4.36378817e+00, 3.72830609e+01, 5.87317358e+00,
       6.16424059e+00, 5.79207132e+00, 3.91161247e+01, 2.88974702e+01,
       3.05911427e+01, 7.30392966e+00, 1.38984444e+01, 1.35462358e+01,
       1.41320552e+01, 8.57915624e+00, 7.99446667e+00, 1.98486646e+01,
       9.70384524e+00, 4.48436667e+00, 2.87258556e+01, 9.70384524e+00,
       9.31613575e+00, 3.60447797e+01, 2.16404670e+01, 2.69765534e+01,
       9.70384524e+00, 1.66898579e+01, 9.39688001e+00, 1.56627684e+01,
       2.56624716e+01, 4.13638561e+01, 1.04387006e+01, 5.35395750e+01,
       1.88694277e+01, 1.54374984e+01, 3.25280079e+01, 1.78326518e+02,
       2.51376620e+01, 1.62699000e+01, 1.75350604e+01, 3.79066013e+01,
       1.33483232e+01, 2.72125170e+01, 6.60027778e+00, 1.03037419e+01,
      

In [33]:
Y_te

350     48.00
1573     4.60
1657    23.50
280     20.00
1149    15.00
        ...  
1939     9.00
1941    17.50
321     90.00
1254    18.12
1830    64.10
Name: modal_price, Length: 448, dtype: float64

In [35]:
df = pd.read_csv("D:/machine learning/datasets/agridata_csv_202110311352.csv")

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836977 entries, 0 to 836976
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   commodity_name  836977 non-null  object 
 1   state           836977 non-null  object 
 2   district        836977 non-null  object 
 3   market          836977 non-null  object 
 4   min_price       836976 non-null  float64
 5   max_price       836976 non-null  float64
 6   modal_price     836976 non-null  float64
 7   date            836976 non-null  object 
dtypes: float64(3), object(5)
memory usage: 51.1+ MB


In [37]:
df.head()

,commodity_name,state,district,market,min_price,max_price,modal_price,date
0,Ajwan,Gujarat,Amreli,Amreli,63.25,79.55,79.55,22-05-19
1,Ajwan,Gujarat,Banaskanth,Dhanera,80.05,80.05,80.05,22-05-19
2,Ajwan,Gujarat,Jamnagar,Jamnagar,90.00,155.00,122.50,22-05-19
3,Ajwan,Gujarat,Mehsana,Kadi,94.50,112.50,107.00,22-05-19
4,Ajwan,Gujarat,Mehsana,Unjha,21.25,148.50,99.75,22-05-19


In [38]:
df = df.dropna()

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836976 entries, 0 to 836976
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   commodity_name  836976 non-null  object 
 1   state           836976 non-null  object 
 2   district        836976 non-null  object 
 3   market          836976 non-null  object 
 4   min_price       836976 non-null  float64
 5   max_price       836976 non-null  float64
 6   modal_price     836976 non-null  float64
 7   date            836976 non-null  object 
dtypes: float64(3), object(5)
memory usage: 57.5+ MB


In [40]:
X = df.iloc[:,:-2]

,commodity_name,state,district,market,min_price,max_price
0,Ajwan,Gujarat,Amreli,Amreli,63.25,79.55
1,Ajwan,Gujarat,Banaskanth,Dhanera,80.05,80.05
2,Ajwan,Gujarat,Jamnagar,Jamnagar,90.00,155.00
3,Ajwan,Gujarat,Mehsana,Kadi,94.50,112.50
4,Ajwan,Gujarat,Mehsana,Unjha,21.25,148.50
...,...,...,...,...,...,...
836972,Arhar (Tur/Red Gram)(Whole),Gujarat,Bharuch,Jambusar(Kaavi),5100.00,5500.00
836973,Arhar (Tur/Red Gram)(Whole),Gujarat,Rajkot,Jasdan,3500.00,5500.00
836974,Arhar (Tur/Red Gram)(Whole),Gujarat,Rajkot,Rajkot,5300.00,6225.00
836975,Arhar (Tur/Red Gram)(Whole),Gujarat,Junagarh,Visavadar,4750.00,5750.00


In [42]:
X['commodity_name'] = le.fit_transform(X['commodity_name'])
X['state'] = le.fit_transform(X['state'])
X['district'] = le.fit_transform(X['district'])
X['market'] = le.fit_transform(X['market'])

In [45]:
Y = df['modal_price']

In [48]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [50]:
rf.fit(X_train,Y_train)

RandomForestRegressor()

In [51]:
rf.score(X_test,Y_test)

0.9707971383764712

In [54]:
model = xgb_model()

In [55]:
model.fit(X_train,Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.7500000000000002,
                              enable_categorical=False, gamma=0, gpu_id=-1,
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.1, max_delta_step=0, max_depth=13,
                              min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=8, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=0.05,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [56]:
model.score(X_test,Y_test)

0.9391934158044104

In [59]:
le_c = LabelEncoder()
le_s = LabelEncoder()
le_d = LabelEncoder()
le_m = LabelEncoder()

In [60]:
deploy = {'model':rf , 'le_c': le_c,'le_s':le_s,'le_d':le_d,'le_m':le_m}
with open('saved_steps.pkl','wb') as file:
    pickle.dump(deploy,file)

In [61]:
with open('saved_steps.pkl','rb') as file:
    mod = pickle.load(file)

In [78]:
df['market'].unique()

array(['Amreli', 'Dhanera', 'Jamnagar', ..., 'Hingoli(Kanegoan Naka)',
       'Jalgaon Jamod(Aasalgaon)', 'Nagarkurnool'], dtype=object)